In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/woke-odds')
print(os.getcwd())

/content/drive/MyDrive/woke-odds


In [ ]:
import pandas as pd

sft_results = pd.read_csv('SFT_test_edit.csv')
dpo_results = pd.read_csv('DPO_test_edit.csv')

In [ ]:
results_df = pd.concat([sft_results, dpo_results], axis=1)

In [ ]:
first_df = pd.concat([sft_results, dpo_results], axis=1)
first_df = first_df[['query', 'SFT_response', 'DPO_response', 'category', 'subclass', 'original_question']]

In [ ]:
# subclass 분포 확인
subclass_counts = first_df['subclass'].value_counts()

print("=== Subclass 분포 ===")
print(subclass_counts)

=== Subclass 분포 ===
subclass
UNF      26
WHOM     20
LEX      20
WHEN     19
WHAT     19
WHERE    17
CONT     15
SEM      14
Name: count, dtype: int64


In [ ]:
results_df = results_df[['query', 'SFT_response', 'DPO_response', 'category', 'subclass', 'original_question']]

print(f"총 샘플 수: {len(results_df)}")
print(f"컬럼: {results_df.columns.tolist()}")

총 샘플 수: 150
컬럼: ['query', 'SFT_response', 'DPO_response', 'category', 'subclass', 'original_question']


In [ ]:
results_df.head()

,query,SFT_response,DPO_response,category,subclass,original_question
0,[AO|WHEN] Who is known as the father of the sc...,NONE,When are you referring to the father of the sc...,AO,WHEN,Who is known as the father of the science?
1,[EM|UNF] Is the growth habit of scillormidium ...,Are you referring to an unfamiliar species or ...,"Does the ""scillorm"" refer to the organism ""Sca...",EM,UNF,Is the growth habit of scillormidium self-supp...
2,[AO|WHOM] Create a survey that asks about eati...,NONE,Who are we asking about in this survey?,AO,WHOM,Create a survey that asks about eating and coo...
3,[AO|WHAT] Give me a list of golf clubs you sho...,NONE,WHAT is the list of golf clubs?,AO,WHAT,Give me a list of golf clubs you should carry ...
4,[AO|WHEN] Who developed an explanation for the...,NONE,WHAT does an explanation include a contradiction?,AO,WHEN,Who developed an explanation for the photoelec...


In [ ]:
sft_none_count = (results_df['SFT_response'] == 'NONE').sum()
dpo_none_count = (results_df['DPO_response'] == 'NONE').sum()

print(f"SFT_response에서 'NONE': {sft_none_count}개")
print(f"DPO_response에서 'NONE': {dpo_none_count}개")
print(f"\n총 샘플 수: {len(results_df)}")
print(f"SFT 'NONE' 비율: {sft_none_count/len(results_df)*100:.1f}%")
print(f"DPO 'NONE' 비율: {dpo_none_count/len(results_df)*100:.1f}%")

SFT_response에서 'NONE': 48개
DPO_response에서 'NONE': 40개

총 샘플 수: 150
SFT 'NONE' 비율: 32.0%
DPO 'NONE' 비율: 26.7%


In [ ]:
both_none = (results_df['SFT_response'] == 'NONE') & (results_df['DPO_response'] == 'NONE')
both_none_count = both_none.sum()

print(both_none_count)

20


In [ ]:
results_df = results_df[~both_none].reset_index(drop=True)

print(f"삭제 후 남은 샘플 수: {len(results_df)}")

sft_none_count = (results_df['SFT_response'] == 'NONE').sum()
dpo_none_count = (results_df['DPO_response'] == 'NONE').sum()

print(f"SFT_response에서 'NONE': {sft_none_count}개")
print(f"DPO_response에서 'NONE': {dpo_none_count}개")
print(f"\n총 샘플 수: {len(results_df)}")
print(f"SFT 'NONE' 비율: {sft_none_count/len(results_df)*100:.1f}%")
print(f"DPO 'NONE' 비율: {dpo_none_count/len(results_df)*100:.1f}%")

삭제 후 남은 샘플 수: 130
SFT_response에서 'NONE': 28개
DPO_response에서 'NONE': 20개

총 샘플 수: 130
SFT 'NONE' 비율: 21.5%
DPO 'NONE' 비율: 15.4%


In [ ]:
!pip install judges transformers torch accelerate bitsandbytes

In [ ]:
! git config --global credential.helper store

In [ ]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `NLP_HW` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/hugging

In [ ]:
!hf auth whoami

user:  jyering


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"

In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True) # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
system_prompt = 'You are an an exact judge assessing clarification questions designed to resolve ambiguity in user queries. Your task is to compare two clarification questions and determine which one is better.'

In [ ]:
user_prompt = '''Given a user question, its ambiguity type, and two clarification questions, determine which question better helps resolve the ambiguity.

Ambiguity types were classified according to the criteria below:
- EM: Questions with unfamiliar entities or self-contradictions
  - UNF: Query contains unfamiliar entities or facts
  - CONT: Query contains self-contradictions

- LA: Questions with lexical or semantic ambiguity
  - LEX: Query contains terms with multiple meanings
  - SEM: Query lacks context leading to multiple interpretations

- AO: Questions with missing contextual information causing confusion
  - WHOM: Query output contains confusion due to missing personal elements
  - WHEN: Query output contains confusion due to missing temporal elements
  - WHERE: Query output contains confusion due to missing spatial elements
  - WHAT: Query output contains confusion due to missing task-specific elements

Task:
User question: {original_question}
Ambiguity type: {category} - {subclass}

Question 1: {SFT_response}
Question 2: {DPO_response}

Evaluation Criteria:
1. Clarity: Does the question directly address and help resolve the specific type of ambiguity identified?
2. Conciseness: Is the question brief and focused without redundant information?
3. Relevance: Does the question align with the user's original intent and the ambiguity type?

Determine which clarification question is superior based on the evaluation criteria above.

Output Format:
Respond with ONLY "Question 1" or "Question 2". Do NOT include any explanation, reasoning, or additional text.
'''

In [ ]:
import torch

def pairwise_comparison(model, tokenizer, original_question, question_1, question_2, category, subclass):
  formatted_user_prompt = user_prompt.format(
      original_question=original_question,
      SFT_response=question_1,
      DPO_response=question_2,
      category=category,
      subclass=subclass
  )

  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": formatted_user_prompt}
  ]

  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )

  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  with torch.no_grad():
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=50,
        temperature=0.1
    )

  generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  return response

In [ ]:
from tqdm import tqdm
#전체 데이터셋 평가
results = []

for idx, row in tqdm(results_df.iterrows(), total=len(results_df)):
    result = pairwise_comparison(
        model=model,
        tokenizer=tokenizer,
        original_question=row['original_question'],
        question_1=row['SFT_response'],
        question_2=row['DPO_response'],
        category=row['category'],
        subclass=row['subclass']
    )

    results.append({
        'query': row['query'],
        'original_question': row['original_question'],
        'category': row['category'],
        'subclass': row['subclass'],
        'question_1': row['SFT_response'],
        'question_2': row['DPO_response'],
        'winner': result
    })

DPO_SFT_results = pd.DataFrame(results)

100%|██████████| 130/130 [01:12<00:00,  1.80it/s]


In [ ]:
DPO_SFT_results.to_csv('DPO_SFT_results.csv', index=False, encoding='utf-8-sig')

In [ ]:
DPO_SFT_results.head()

,query,original_question,category,subclass,question_1,question_2,winner
0,[AO|WHEN] Who is known as the father of the sc...,Who is known as the father of the science?,AO,WHEN,NONE,When are you referring to the father of the sc...,Question 2
1,[EM|UNF] Is the growth habit of scillormidium ...,Is the growth habit of scillormidium self-supp...,EM,UNF,Are you referring to an unfamiliar species or ...,"Does the ""scillorm"" refer to the organism ""Sca...",Question 1
2,[AO|WHOM] Create a survey that asks about eati...,Create a survey that asks about eating and coo...,AO,WHOM,NONE,Who are we asking about in this survey?,Question 2
3,[AO|WHAT] Give me a list of golf clubs you sho...,Give me a list of golf clubs you should carry ...,AO,WHAT,NONE,WHAT is the list of golf clubs?,Question 2
4,[AO|WHEN] Who developed an explanation for the...,Who developed an explanation for the photoelec...,AO,WHEN,NONE,WHAT does an explanation include a contradiction?,Question 1


In [ ]:
import pandas as pd

final_results = pd.read_csv('DPO_SFT_results_edited.csv')

In [ ]:
final_results.head()

,query,original_question,category,subclass,question_1,question_2,winner
0,[AO|WHEN] Who is known as the father of the sc...,Who is known as the father of the science?,AO,WHEN,NONE,When are you referring to the father of the sc...,Question 2
1,[EM|UNF] Is the growth habit of scillormidium ...,Is the growth habit of scillormidium self-supp...,EM,UNF,Are you referring to an unfamiliar species or ...,"Does the ""scillorm"" refer to the organism ""Sca...",Question 1
2,[AO|WHOM] Create a survey that asks about eati...,Create a survey that asks about eating and coo...,AO,WHOM,NONE,Who are we asking about in this survey?,Question 2
3,[AO|WHAT] Give me a list of golf clubs you sho...,Give me a list of golf clubs you should carry ...,AO,WHAT,NONE,WHAT is the list of golf clubs?,Question 2
4,[AO|WHEN] Who developed an explanation for the...,Who developed an explanation for the photoelec...,AO,WHEN,NONE,WHAT does an explanation include a contradiction?,Question 2


In [ ]:
final_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   query              115 non-null    object
 1   original_question  115 non-null    object
 2   category           115 non-null    object
 3   subclass           115 non-null    object
 4   question_1         115 non-null    object
 5   question_2         115 non-null    object
 6   winner             115 non-null    object
dtypes: object(7)
memory usage: 6.4+ KB


In [ ]:
# winner 컬럼의 고유값 개수 세기
winner_counts = final_results['winner'].value_counts()

print("=== Winner 분포 ===")
print(winner_counts)
print(f"\n총 샘플 수: {len(final_results)}")
print(f"\n비율:")
for winner, count in winner_counts.items():
    print(f"  {winner}: {count}개 ({count/len(final_results)*100:.1f}%)")

=== Winner 분포 ===
winner
Question 2    56
Question 1    51
SAME           8
Name: count, dtype: int64

총 샘플 수: 115

비율:
  Question 2: 56개 (48.7%)
  Question 1: 51개 (44.3%)
  SAME: 8개 (7.0%)


In [ ]:
# subclass 분포 확인
subclass_counts = final_results['subclass'].value_counts()

print("=== Subclass 분포 ===")
print(subclass_counts)

=== Subclass 분포 ===
subclass
UNF      22
WHEN     15
WHAT     15
CONT     15
WHERE    13
LEX      13
SEM      12
WHOM     10
Name: count, dtype: int64


In [ ]:
from scipy.stats import binomtest

# DPO가 우월한지 검정 (귀무가설: 50-50)
result = binomtest(56, 56+51, 0.5, alternative='greater')
p_value = result.pvalue

print(f"p-value: {p_value:.4f}")
print(f"통계적 유의성: {'유의미함 (p < 0.05)' if p_value < 0.05 else '유의미하지 않음 (p >= 0.05)'}")

p-value: 0.3496
통계적 유의성: 유의미하지 않음 (p >= 0.05)


In [ ]:
import pandas as pd

# Subclass별 Winner 분포
subclass_winner = pd.crosstab(
    final_results['subclass'],
    final_results['winner'],
    margins=True
)

print("=== Subclass별 Winner 분포 ===")
print(subclass_winner)

# Subclass별 승률 계산
print("\n=== Subclass별 DPO 승률 ===")
for subclass in final_results['subclass'].unique():
    subclass_data = final_results[results_df['subclass'] == subclass]
    total = len(subclass_data)

    dpo_wins = (subclass_data['winner'] == 'Question 2').sum()
    sft_wins = (subclass_data['winner'] == 'Question 1').sum()
    same = (subclass_data['winner'] == 'SAME').sum()

    dpo_win_rate = dpo_wins / total * 100

    print(f"\n{subclass}:")
    print(f"  총 {total}개")
    print(f"  DPO 승: {dpo_wins}개 ({dpo_win_rate:.1f}%)")
    print(f"  SFT 승: {sft_wins}개 ({sft_wins/total*100:.1f}%)")
    print(f"  SAME: {same}개 ({same/total*100:.1f}%)")

    # DPO 우세 여부 표시
    if dpo_win_rate > 50:
        print(f"  → DPO 우세 ✅")
    elif dpo_win_rate < 50:
        print(f"  → SFT 우세 ⚠️")
    else:
        print(f"  → 동등")

=== Subclass별 Winner 분포 ===
winner    Question 1  Question 2  SAME  All
subclass                                   
CONT               3           5     7   15
LEX                9           4     0   13
SEM                8           3     1   12
UNF               11          11     0   22
WHAT               4          11     0   15
WHEN               6           9     0   15
WHERE              4           9     0   13
WHOM               6           4     0   10
All               51          56     8  115

=== Subclass별 DPO 승률 ===

WHEN:
  총 18개
  DPO 승: 14개 (77.8%)
  SFT 승: 2개 (11.1%)
  SAME: 2개 (11.1%)
  → DPO 우세 ✅

UNF:
  총 21개
  DPO 승: 9개 (42.9%)
  SFT 승: 11개 (52.4%)
  SAME: 1개 (4.8%)
  → SFT 우세 ⚠️

WHOM:
  총 12개
  DPO 승: 4개 (33.3%)
  SFT 승: 7개 (58.3%)
  SAME: 1개 (8.3%)
  → SFT 우세 ⚠️

WHAT:
  총 13개
  DPO 승: 9개 (69.2%)
  SFT 승: 3개 (23.1%)
  SAME: 1개 (7.7%)
  → DPO 우세 ✅

LEX:
  총 12개
  DPO 승: 3개 (25.0%)
  SFT 승: 7개 (58.3%)
  SAME: 2개 (16.7%)
  → SFT 우세 ⚠️

WHERE:
  총 13개
  DPO 승: 5개

/tmp/ipython-input-1575647931.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subclass_data = final_results[results_df['subclass'] == subclass]
/tmp/ipython-input-1575647931.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subclass_data = final_results[results_df['subclass'] == subclass]
/tmp/ipython-input-1575647931.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subclass_data = final_results[results_df['subclass'] == subclass]
/tmp/ipython-input-1575647931.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subclass_data = final_results[results_df['subclass'] == subclass]
/tmp/ipython-input-1575647931.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subclass_data = final_results[results_df['subclass'] == subclass]
/tmp/ipython-input-1575647931.py:16: UserWarning: Boolean Series key will be reindexed to match

In [ ]:
# 후처리 전후 분포
before = {
    'UNF': 26, 'WHEN': 19, 'WHAT': 19, 'WHOM': 20,
    'LEX': 20, 'WHERE': 17, 'CONT': 15, 'SEM': 14
}

after = {
    'UNF': 22, 'WHEN': 15, 'WHAT': 15, 'CONT': 15,
    'WHERE': 13, 'LEX': 13, 'SEM': 12, 'WHOM': 10
}

# 차이 계산
print("=== 후처리로 삭제된 데이터 (두 모델 모두 어려운 유형) ===\n")

differences = []
for subclass in before.keys():
    deleted = before[subclass] - after[subclass]
    deletion_rate = (deleted / before[subclass]) * 100
    differences.append({
        'subclass': subclass,
        'before': before[subclass],
        'after': after[subclass],
        'deleted': deleted,
        'deletion_rate': deletion_rate
    })

# 삭제 개수 기준 정렬
differences.sort(key=lambda x: x['deleted'], reverse=True)

total_before = sum(before.values())
total_deleted = sum(d['deleted'] for d in differences)

for d in differences:
    print(f"{d['subclass']:8s}: {d['before']:2d} → {d['after']:2d}  "
          f"(-{d['deleted']}개, {d['deletion_rate']:.1f}% 삭제)")

print(f"\n{'='*60}")
print(f"전체: {total_before} → {sum(after.values())}  "
      f"(-{total_deleted}개, {total_deleted/total_before*100:.1f}% 삭제)")

=== 후처리로 삭제된 데이터 (두 모델 모두 어려운 유형) ===

WHOM    : 20 → 10  (-10개, 50.0% 삭제)
LEX     : 20 → 13  (-7개, 35.0% 삭제)
UNF     : 26 → 22  (-4개, 15.4% 삭제)
WHEN    : 19 → 15  (-4개, 21.1% 삭제)
WHAT    : 19 → 15  (-4개, 21.1% 삭제)
WHERE   : 17 → 13  (-4개, 23.5% 삭제)
SEM     : 14 → 12  (-2개, 14.3% 삭제)
CONT    : 15 → 15  (-0개, 0.0% 삭제)

전체: 150 → 115  (-35개, 23.3% 삭제)
